## やりたいこと
proto2020.pyのコードを改良して、起動時の初期値設定処理を実装
そのテスト

In [1]:
"""
リアルタイム・スペクトルアナライザー
"""

#%%
# cording = UTF-8

import os,configparser
import scipy,pyaudio
import numpy as np
import matplotlib as plt
from matplotlib.colors import LogNorm

print ("Scipy version:{0}".format(scipy.__version__))
print ("Pyaudio version:{0}".format(pyaudio.__version__))
print ("Numpy version:{0}".format(np.__version__))

###########################グローバル変数の初期値###########################
base_dir = os.path.dirname(__file__)    #カレントディレクトリ

a_index = None  #オーディオインデックス

br = pyaudio.paInt16    #ビットレート
sr = 44100  #サンプリングレート
chunk = 1024  #ストリームサイズならびにFFTの窓幅


Scipy version:1.5.2
Pyaudio version:0.2.11
Numpy version:1.19.2


In [2]:
class initial:
    def __init__(self,base_dir,a_index,br,sr,chunk):
        #グローバル変数の取り込み
        self.base_dir = base_dir
        self.a_index = a_index,
        self.br = br
        self.sr = sr
        self.chunk = chunk
        self.pa = pyaudio.PyAudio()

    #オーディオインデックスの取得
    def set_indexes(self):
        print("***List of available audio devices:***")
        for i in range(self.pa.get_device_count()):
            print(i,self.pa.get_device_info_by_index(i).get("name"),sep = " - ")
        x = int(input("Select Audio device Index No."))
        print("***Selected audio device #{0}.***".format(x))
        return x

    #設定値の書き出し
    def save_ini(self):
        x = configparser.ConfigParser()

        x["General"] = {
            "base_dir":self.base_dir
        }

        x["params"] = {
            "br":self.br,
            "sr":self.sr,
            "chunk":self.chunk
        }

        if self.a_index == None:
            x["params"]["a_index"] = ""
        else:
            x["params"]["a_index"] = str(self.a_index)

        with open(os.path.splitext(
            os.path.basename(__file__))[0] + ".ini","w"
        ) as cfgfile:
            x.write(cfgfile)

    #設定値の読み出し
    def load_ini(self):
        x = configparser.ConfigParser()
        x.read(
            os.path.join(
                base_dir,str(
                    os.path.splitext(
                        os.path.basename(__file__)
                        )[0] + ".ini"
                    )
                )
        )

        #グローバル変数との区別の為全て頭に"l_"をつけて読みだす
        l_base_dir = x.get("General","base_dir")
        l_a_index = x.get("params","a_index")
        if l_a_index =="":
            l_a_index = None
        else:
            l_a_index = int(l_a_index)
        l_br = x.getint("params","br")
        l_sr = x.getint("params","sr")
        l_chunk = x.getint("params","chunk")

        return l_base_dir,l_a_index,l_br,l_sr,l_chunk

    #起動時処理
    def bootup(self):
        #iniファイルの有無判定 存在すればiniファイルから変数を読み出す
        if os.path.exists(
            os.path.join(
                base_dir,str(
                    os.path.splitext(
                        os.path.basename(__file__)
                        )[0] + ".ini"
                    )
                )
            ):
            self.base_dir,self.a_index,self.br,self.sr,self.chunk = self.load_ini()
            print("Loaded initial settings.\n***Audio device #{0} - {1} is selected***"\
            .format(self.a_index,self.pa.get_device_info_by_index(self.a_index)\
                .get("name")))
        else:
            #オーディオインデックスを選択後、変数をiniファイルに格納する
            self.a_index = self.set_indexes()
            self.save_ini()
            print(
                "Saved deffault settings:" + os.path.splitext(
                    os.path.basename(__file__)
                    )[0] + ".ini"
            )
        return self.base_dir,self.a_index,self.br,self.sr,self.chunk



In [3]:
init = initial(base_dir,a_index,br,sr,chunk)
base_dir,a_index,br,sr,chunk = init.bootup()

***List of available audio devices:***
0 - Microsoft Sound Mapper - Input
1 - デスクトップ マイク (Microsoft® LifeCam 
2 - マイク配列 (Realtek(R) Audio)
3 - Microsoft Sound Mapper - Output
4 - SHARP HDMI (インテル(R) ディスプレイ用オーディ
5 - Speakers (Realtek HD Audio output)
6 - マイク配列 1 (Realtek HD Audio Mic input with SST)
7 - マイク配列 2 (Realtek HD Audio Mic input with SST)
8 - マイク配列 3 (Realtek HD Audio Mic input with SST)
9 - Output (インテル(R) ディスプレイ用オーディオ - 出力 1)
10 - デスクトップ マイク (Microsoft® LifeCam HD-3000)
11 - ヘッドホン ()
12 - ヘッドホン ()
13 - ヘッドセット (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(pianist_dannaのAirPods Pro))
14 - ヘッドセット (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(pianist_dannaのAirPods Pro))
15 - ヘッドセット (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(JAMBOX by Jawbone))
16 - ヘッドセット (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(JAMBOX by Jawbone))
17 - Output (@System32\drivers\bthhfenum.sys,#4;%1 Hands-Free HF Audio%0
;(Xperia 10))
18 

オーディオインデックスが設定され、iniファイルへの保存成功  
今回はカレントファイルの名前の変更に伴ってiniファイルの名前も動的に変わるようにした  
  
一度グローバル変数を消去後、再起動する

In [4]:
del base_dir,a_index,br,sr,chunk

In [6]:
"""
リアルタイム・スペクトルアナライザー
"""

#%%
# cording = UTF-8

import os,configparser
import scipy,pyaudio
import numpy as np
import matplotlib as plt
from matplotlib.colors import LogNorm

print ("Scipy version:{0}".format(scipy.__version__))
print ("Pyaudio version:{0}".format(pyaudio.__version__))
print ("Numpy version:{0}".format(np.__version__))

###########################グローバル変数の初期値###########################
base_dir = os.path.dirname(__file__)    #カレントディレクトリ

a_index = None  #オーディオインデックス

br = pyaudio.paInt16    #ビットレート
sr = 44100  #サンプリングレート
chunk = 1024  #ストリームサイズならびにFFTの窓幅


Scipy version:1.5.2
Pyaudio version:0.2.11
Numpy version:1.19.2


In [7]:
init = initial(base_dir,a_index,br,sr,chunk)
base_dir,a_index,br,sr,chunk = init.bootup()

Loaded initial settings.
***Audio device #1 - デスクトップ マイク (Microsoft® LifeCam  is selected***


生成されたiniファイルから設定値が読み出されている  
(コード上の初期値は`a_index = None`だが、起動後は`a_index = 1`になっている  
  
## わかったこと
- proto2020.pyではiniファイル操作と変数読み出しを別関数にしていたが、ここが分離することはない(多分)のでまとめ処理可能だった
- 本格的にコンストラクタを使ったが、インスタンス化の際に面倒な分各メソッドはシンプルになった
  
## 次にやること
- まずはウェーブフォームアニメーションの実装から 将来的に使わん可能性大だが…
- 今回はクラス警鐘をあまり意識する必要はなさそう